<a href="https://colab.research.google.com/github/ZZaman1989/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-17 02:45:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-10-17 02:45:22 (4.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [6]:
# Create DataFrame
review_df = df.filter(df["total_votes"]>= 20)
helpful_votes = review_df.filter(review_df["helpful_votes"]/review_df["total_votes"]>= 0.5)
helpful_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...|2015-08-31 00:00:00|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...| 

In [10]:
paid_df = helpful_votes.filter(helpful_votes["vine"]=="Y")
paid_df.describe().show()

+-------+-----------+--------------------+--------------+----------+--------------------+--------------------+----------------+------------------+-----------------+------------------+----+-----------------+--------------------+--------------------+
|summary|marketplace|         customer_id|     review_id|product_id|      product_parent|       product_title|product_category|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+--------------------+--------------+----------+--------------------+--------------------+----------------+------------------+-----------------+------------------+----+-----------------+--------------------+--------------------+
|  count|         94|                  94|            94|        94|                  94|                  94|              94|                94|               94|                94|  94|               94|                  94|                  94|
|   

In [8]:
unpaid_df = helpful_votes.filter(helpful_votes["vine"]=="N")
unpaid_df.describe().show()

+-------+-----------+-------------------+--------------+-------------------+-------------------+--------------------+----------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+
|summary|marketplace|        customer_id|     review_id|         product_id|     product_parent|       product_title|product_category|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+-------------------+--------------+-------------------+-------------------+--------------------+----------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+
|  count|      40471|              40471|         40471|              40471|              40471|               40471|           40471|             40471|             40471|             40471|40471|            40471|           

In [11]:
paid_number = paid_df.count()
paid_fivestar_number = paid_df[paid_df["star_rating"]==5].count()
paid_fivestar_percentage = paid_fivestar_number/paid_number

In [12]:
print(paid_number)
print(paid_fivestar_number)
print(paid_fivestar_percentage)

94
48
0.5106382978723404


In [13]:
unpaid_number = unpaid_df.count()
unpaid_fivestar_number = unpaid_df[paid_df["star_rating"]==5].count()
unpaid_fivestar_percentage = unpaid_fivestar_number/unpaid_number

In [14]:
print(unpaid_number)
print(unpaid_fivestar_number)
print(unpaid_fivestar_percentage)

40471
15663
0.38701786464381904
